## 情感分析项目

本项目的目标是基于用户提供的评论，通过算法自动去判断其评论是正面的还是负面的情感。比如给定一个用户的评论：
- 评论1： “我特别喜欢这个电器，我已经用了3个月，一点问题都没有！”
- 评论2： “我从这家淘宝店卖的东西不到一周就开始坏掉了，强烈建议不要买，真实浪费钱”

对于这两个评论，第一个明显是正面的，第二个是负面的。 我们希望搭建一个AI算法能够自动帮我们识别出评论是正面还是负面。

情感分析的应用场景非常丰富，也是NLP技术在不同场景中落地的典范。比如对于一个证券领域，作为股民，其实比较关注舆论的变化，这个时候如果能有一个AI算法自动给网络上的舆论做正负面判断，然后把所有相关的结论再整合，这样我们可以根据这些大众的舆论，辅助做买卖的决策。 另外，在电商领域评论无处不在，而且评论已经成为影响用户购买决策的非常重要的因素，所以如果AI系统能够自动分析其情感，则后续可以做很多有意思的应用。 

情感分析是文本处理领域经典的问题。整个系统一般会包括几个模块：
- 数据的抓取： 通过爬虫的技术去网络抓取相关文本数据
- 数据的清洗/预处理：在本文中一般需要去掉无用的信息，比如各种标签（HTML标签），标点符号，停用词等等
- 把文本信息转换成向量： 这也成为特征工程，文本本身是不能作为模型的输入，只有数字（比如向量）才能成为模型的输入。所以进入模型之前，任何的信号都需要转换成模型可识别的数字信号（数字，向量，矩阵，张量...)
- 选择合适的模型以及合适的评估方法。 对于情感分析来说，这是二分类问题（或者三分类：正面，负面，中性），所以需要采用分类算法比如逻辑回归，朴素贝叶斯，神经网络，SVM等等。另外，我们需要选择合适的评估方法，比如对于一个应用，我们是关注准确率呢，还是关注召回率呢？ 

在本次项目中，我们已经给定了训练数据和测试数据，它们分别是 ``train.positive.txt``, ``train.negative.txt``， ``test_combined.txt``. 请注意训练数据和测试数据的格式不一样，详情请见文件内容。 整个项目需要完成以下步骤：

数据的读取以及清洗： 从给定的.txt中读取内容，并做一些数据清洗，这里需要做几个工作： 
- （1） 文本的读取，需要把字符串内容读进来。 
- （2）去掉无用的字符比如标点符号，多余的空格，换行符等 
- （3） 把文本转换成``TF-IDF``向量： 这部分直接可以利用sklearn提供的``TfidfVectorizer``类来做。
- （4） 利用逻辑回归等模型来做分类，并通过交叉验证选择最合适的超参数

项目中需要用到的数据：
- ``train.positive.txt``, ``train.negative.txt``， ``test_combined.txt``： 训练和测试数据
- ``stopwords.txt``： 停用词库

### 1. File Reading: 文本读取 

In [1]:
import pandas as pd
import numpy as np
import re
import jieba

In [2]:
# 定义成全局变量
train_comments = []
train_labels = []
test_comments = []
test_labels = []

In [3]:
def process_file():
    """
    读取训练数据和测试数据，并对它们做一些预处理
    """    
    train_pos_file = "data/train_positive.txt"
    train_neg_file = "data/train_negative.txt"
    test_comb_file = "data/test_combined.txt"
    
    # TODO: 读取文件部分，把具体的内容写入到变量里面
    # 读取文件
    with open(train_pos_file) as f:
        string_tp = f.read()
    with open(train_neg_file) as f:
        string_tn = f.read()
    with open(test_comb_file) as f:
        string_test = f.read()
    
    # 将训练数据和训练标签进行写入
    string_tp_content = re.findall(r"<review .*?>(.*?)</review>", string_tp, re.S)
    string_tn_content = re.findall(r"<review .*?>(.*?)</review>", string_tn, re.S)
    string_test_content = re.findall('<review .*? label="(.+?)">(.*?)</review>', string_test, re.S|re.M)
    for i in range(len(string_tp_content)):
        temp = string_tp_content[i].strip() # 将内容两边的空格删除
        temp = re.sub(r"\n", " ", temp) # 将内容中间的\n符换成空格
        train_comments.append(temp)
        train_labels.append(1)  # 正面情绪使用1作为标签添加
    for i in range(len(string_tn_content)):
        temp = string_tn_content[i].strip() # 将内容两边的空格删除
        temp = re.sub(r"\n", " ", temp) # 将内容中间的\n符换成空格
        train_comments.append(temp)
        train_labels.append(0)  # 负面情绪使用0作为标签添加
    for i in range(len(string_test_content)):
        temp_content = string_test_content[i][1].strip() # 将内容两边的空格删除
        temp_content = re.sub(r"\n", " ", temp_content) # 将内容中间的\n符换成空格
        temp_target = int(string_test_content[i][0]) # 将匹配出来的标签转化为int型
        test_comments.append(temp_content) 
        test_labels.append(temp_target)  # 将匹配出来的标签项添加到test_labels中去

In [4]:
if __name__ == "__main__":
    process_file()

### 2. Explorary Analysis: 做一些简单的可视化分析

In [5]:
# 训练数据和测试数据大小
print (len(train_comments),len(train_labels), len(test_comments), len(test_labels))

8065 8065 2500 2500


In [17]:
type(train_labels)

list

> 这里有一个假设想验证。我觉得，如果一个评论是负面的，则用户留言时可能会长一些，因为对于负面的评论，用户很可能会把一些细节写得很清楚。但对于正面的评论，用户可能就只写“非常好”，这样的短句。我们想验证这个假设。 为了验证这个假设，打算画两个直方图，分别对正面的评论和负面的评论。 具体的做法是：1. 把正面和负面评论分别收集，之后分别对正面和负面评论画一个直方图。 2.  直方图的X轴是评论的长度，所以从是小到大的顺序。然后Y轴是对于每一个长度，出现了多少个正面或者负面的评论。 通过两个直方图的比较，即可以看出``评论``是否是一个靠谱的特征。


In [ ]:
# TODO: 对于训练数据中的正负样本，分别画出一个histogram， histogram的x抽是每一个样本中字符串的长度，y轴是拥有这个长度的样本的百分比。
#       并说出样本长度是否对情感有相关性 (需要先用到结巴分词)
#       参考：https://baike.baidu.com/item/%E7%9B%B4%E6%96%B9%E5%9B%BE/1103834?fr=aladdin
#       画饼状图参考： https://pythonspot.com/matplotlib-histogram/   
#                   https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist.html




### 3. 文本预处理
> 在此部分需要做文本预处理方面的工作。 分为几大块：
- ``去掉特殊符号``  比如#$.... 
- ``把数字转换成特殊单词`` 把数字转换成 " NUM "。 注意：NUM前面和后面加一个空格，这样可以保证之后分词时被分掉。
- ``分词并过滤掉停用词`` 停用词库已经提供，需要读取停用词库，并按照此停用词库做过滤。 停用词库使用给定的文件：``stopwords.txt`` 

In [6]:
def clean_symbols(text):
    """
    对特殊符号做一些处理，此部分已写好。如果不满意也可以自行改写，不记录分数。
    """
    text = re.sub('[!！]+', "!", text)
    text = re.sub('[?？]+', "?", text)
    text = re.sub("[a-zA-Z#$%&\'()*+,-./:;：<=>@，。★、…【】《》“”‘’[\\]^_`{|}~]+", " OOV ", text)
    return re.sub("\s+", " ", text)  

# TODO：对于train_comments, test_comments进行字符串的处理，几个考虑的点：
#   1. 去掉特殊符号
#   2. 把数字转换成特殊字符或者单词
#   3. 分词并做停用词过滤
#   4. ... （或者其他）
#
#   需要注意的点是，由于评论数据本身很短，如果去掉的太多，很可能字符串长度变成0
#   预处理部分，可以自行选择合适的方.

def change_number(text):
    """
    对评论中出现的数字，一律替换成" NUM "。
    """
    text = re.sub(r'[0-9]+', " NUM ", text)
    return text

train_comments_cleaned = [] 
test_comments_cleaned = []

In [ ]:
# 处理训练数据
for i in train_comments:
    temp = clean_symbols(i)
    temp = change_number(temp)
    train_comments_cleaned.append(temp)
# 处理测试数据
for i in test_comments:
    temp = clean_symbols(i)
    temp = change_number(temp)
    test_comments_cleaned.append(temp)

In [8]:
train_comments_cleaned_original = train_comments_cleaned
test_comments_cleaned_original = test_comments_cleaned

In [9]:
# 加载停用词库
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  

#对词进行分词并去除停用词
def seg_sentence(sentence):  
    sentence_seged = jieba.cut(sentence)  
    stopwords = stopwordslist('data/stopwords.txt')  # 这里加载停用词的路径  
    outstr = '' 
    # 对分词后的内容进行循环，再与停用词库进行对比筛选
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "  
    return outstr 

In [10]:
train_comments_cleaned = []
test_comments_cleaned = []

In [ ]:
# 先对训练集进行分词以及去停用词
for words in train_comments_cleaned_original:
    temp = seg_sentence(words)    # 分词并去除停用词
    train_comments_cleaned.append(temp)    # train_comments_cleaned就是分词去除停用词

In [ ]:
# 再对测试集进行分词以及去停用词
for words in test_comments_cleaned_original:
    temp = seg_sentence(words)    # 去除停用词
    test_comments_cleaned.append(temp)    # test_comments_cleaned就是分词去除停用次

In [14]:
# 打印一下看看
print (train_comments_cleaned[0], test_comments_cleaned[0])
print (train_comments_cleaned[1], test_comments_cleaned[1])

请问 这机 不是 有个 遥控器  终于 找到 同道中人 初中   OOV   已经 喜欢 上   OOV   同学 都 鄙夷 眼光 看   OOV   人为   OOV   样子 古怪 说 ＂ 丑 ＂ 当场 气晕 现在 同道中人   OOV   好开心 !   OOV   !   OOV   
发短信 特别 不 方便 ! 背后 屏幕 很大 起来 不 舒服   OOV   手触 屏 ! 切换 屏幕 很 麻烦 !  看 完 已 深夜 两点   OOV   却 坐在 电脑前 情难 自禁   OOV   这是 最好 结局   OOV   惟有   OOV   就让 前世 今生 纠结 停留 此刻   OOV   再 相逢 时   OOV   愿 人生 不再 人 唏嘘   OOV   身心 会 只 居 一处   OOV   痛心 人   OOV   爱   OOV   


### 4. 把文本转换成向量
> 预处理好文本之后，我们就需要把它转换成向量形式，这里我们使用tf-idf的方法。 sklearn自带此功能，直接调用即可。输入就是若干个文本，输出就是每个文本的tf-idf向量。详细的使用说明可以在这里找到： 参考：https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html  这里需要特别注意的一点是：对于训练数据调用fit_transform, 也就是训练的过程。 但对于测试数据，不能再做训练，而是直接使用已经训练好的object做transform操作。

In [19]:
# TODO: 利用tf-idf从文本中提取特征,写到数组里面. 
#       参考：https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

from sklearn.feature_extraction.text import TfidfVectorizer
from time import time
# from sklearn.model_selection import train_test_split
t0 = time()
vectorizer = TfidfVectorizer()
print("生成tfidf的时间为：{}".format(time() - t0))
X_train = vectorizer.fit_transform(train_comments_cleaned) # 训练数据的特征
y_train = train_labels # 训练数据的label
X_test = vectorizer.transform(test_comments_cleaned)  # 测试数据的特征
y_test = test_labels  # 测试数据的label

print (np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test))

生成tfidf的时间为：%
(8065, 23101) (2500, 23101) (8065,) (2500,)


### 5. 通过交叉验证来训练模型
> 接下来需要建模了！ 这里我们分别使用逻辑回归，朴素贝叶斯和SVM来训练。针对于每一个方法我们使用交叉验证（gridsearchCV)， 并选出最好的参数组合，然后最后在测试数据上做验证。 

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [23]:
from sklearn.linear_model import LogisticRegression
# TODO： 利用逻辑回归来训练模型
#       1. 评估方式： F1-score
#       2. 超参数（hyperparater）的选择利用grid search https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#       3. 打印出在测试数据中的最好的结果（precision, recall, f1-score, 需要分别打印出正负样本，以及综合的）
#       请注意：做交叉验证时绝对不能用测试数据。 测试数据只能用来最后的”一次性“检验。
#       逻辑回归的使用方法请参考：http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
#       对于逻辑回归，经常调整的超参数为： C

# 交叉验证寻找最优参数C
params_c = np.logspace(-4,1,11)
lr = LogisticRegression(penalty='l1')
grid_search = GridSearchCV(lr,param_grid={'C':params_c},cv=5,scoring='f1')
grid_search.fit(X_train,y_train)
c_best = grid_search.best_params_['C']

# 利用交叉验证获得的C重新训练lr模型
lr_clf = LogisticRegression(penalty='l1',C=c_best)
lr_clf.fit(X_train,y_train)

# 打印报道
print(classification_report(y_test,lr_clf.predict(X_test)))

/Users/jason/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jason/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.83      0.48      0.61      1250
           1       0.64      0.90      0.75      1250

    accuracy                           0.69      2500
   macro avg       0.73      0.69      0.68      2500
weighted avg       0.73      0.69      0.68      2500



In [25]:
from sklearn.naive_bayes import MultinomialNB
# TODO： 利用朴素贝叶斯来训练模型
#       1. 评估方式： F1-score
#       2. 超参数（hyperparater）的选择利用grid search https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#       3. 打印出在测试数据中的最好的结果（precision, recall, f1-score, 需要分别打印出正负样本，以及综合的）
#       请注意：做交叉验证时绝对不能用测试数据。 测试数据只能用来最后的”一次性“检验。
#       朴素贝叶斯的使用方法请参考：https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB
#       对于朴素贝叶斯，一般不太需要超参数的调节。但如果想调参，也可以参考上面的链接，有几个参数是可以调节的。 

#直接使用朴素贝叶斯训练
mnb_clf = MultinomialNB()
mnb_clf.fit(X_train,y_train)
print(classification_report(y_test, mnb_clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.92      0.30      0.45      1250
           1       0.58      0.98      0.73      1250

    accuracy                           0.64      2500
   macro avg       0.75      0.64      0.59      2500
weighted avg       0.75      0.64      0.59      2500



In [27]:
from sklearn.svm import SVC
# TODO： 利用SVM来训练模型
#       1. 评估方式： F1-score
#       2. 超参数（hyperparater）的选择利用grid search https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#       3. 打印出在测试数据中的最好的结果（precision, recall, f1-score, 需要分别打印出正负样本，以及综合的）
#       请注意：做交叉验证时绝对不能用测试数据。 测试数据只能用来最后的”一次性“检验。
#       SVM的使用方法请参考：http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
#       对于SVM模型，经常调整的超参数为：C, gamma, kernel。 这里的参数C跟逻辑回归是一样的， gamma和kernel是针对于SVM的参数
#       在这里先不要考虑他们的含义（或者通过官方文档试图理解一下）， 在课程最后的部分会讲到这些内容。 

# 对svm进行参数调优
params_c = {'C': [1e3, 1e4, 1e5],
            'gamma': [0.0001, 0.005, 0.1] }
svm_clf = GridSearchCV(SVC(kernel='rbf'),params_c,cv=5,scoring='f1')
svm_clf.fit(X_train,y_train)
best_params = svm_clf.best_params_

In [30]:
svc_clf = SVC(kernel='rbf', C=best_params['C'], gamma=best_params['gamma'])
svc_clf.fit(X_train,y_train)
print(classification_report(y_test,svc_clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.60      0.69      1250
           1       0.68      0.86      0.76      1250

    accuracy                           0.73      2500
   macro avg       0.75      0.73      0.73      2500
weighted avg       0.75      0.73      0.73      2500



> 对于超参数的调整，我们经常使用gridsearch，这也是工业界最常用的方法，但它的缺点是需要大量的计算，所以近年来这方面的研究也成为了重点。 其中一个比较经典的成果为Bayesian Optimization（利用贝叶斯的思路去寻找最好的超参数）。Ryan P. Adams主导的Bayesian Optimization利用高斯过程作为后验概率（posteior distribution）来寻找最优解。 https://papers.nips.cc/paper/4522-practical-bayesian-optimization-of-machine-learning-algorithms.pdf 在下面的练习中，我们尝试使用Bayesian Optimization工具来去寻找最优的超参数。参考工具：https://github.com/fmfn/BayesianOptimization  感兴趣的朋友可以去研究一下。 